In [30]:
import pandas as pd

In [31]:
import numpy as np

In [32]:
data_name = "_.nii.gz"

mask_name = "_.nii.gz"

In [45]:
data_path =  r"../database/retest_data/data"

In [46]:
dataa_path = r"../database/retest_data/label"

In [56]:
outputImg_path = r"./seventest/data"

In [55]:
outputMask_path = r"./seventest/label"

In [37]:
import os

In [57]:
if not os.path.exists(outputImg_path):
    os.mkdir(outputImg_path)
if not os.path.exists(outputMask_path):
    os.mkdir(outputMask_path)

In [39]:
pathhgg_list = []
pathlgg_list = []

In [50]:
def file_name_path(file_dir, dir=True, file=False):
    """
    get root path,sub_dirs,all_sub_files
    :param file_dir:
    :return: dir or file
    """
    mri_list = os.listdir(file_dir)
    seg_list = list(map(lambda x: x.replace('volume', 'segmentation').replace('.nii', '.nii'), mri_list))

    mri_list = list(map(lambda x: os.path.join(file_dir, x), mri_list))
    seg_list = list(map(lambda x: os.path.join(bratslgg_path, x), seg_list))
    return mri_list , seg_list

In [51]:
pathmri_list,pathmask_list = file_name_path(data_path)
# pathmri_list.remove('./database/retest_data/label/.ipynb_checkpoints')
# pathmask_list.remove('./database/re/label/.ipynb_checkpoints')
print(pathmri_list)
print("------------")
print(pathmask_list)

['../database/retest_data/data/3GP1RSTO.nii.gz', '../database/retest_data/data/3CT52UTW.nii.gz', '../database/retest_data/data/1RYHNWAY.nii.gz', '../database/retest_data/data/2WUKD0FS.nii.gz', '../database/retest_data/data/2KEL021V.nii.gz', '../database/retest_data/data/0ESRE0DB.nii.gz', '../database/retest_data/data/0ET3UZSR.nii.gz', '../database/retest_data/data/2KWIPQOS.nii.gz', '../database/retest_data/data/0QCYIPQQ.nii.gz', '../database/retest_data/data/1KCPYDVM.nii.gz', '../database/retest_data/data/4DEQNVOE.nii.gz', '../database/retest_data/data/1TCAPQBI.nii.gz', '../database/retest_data/data/0JNTNNBN.nii.gz', '../database/retest_data/data/4GC2RPXG.nii.gz', '../database/retest_data/data/1FRWVMWS.nii.gz', '../database/retest_data/data/1R0TXANO.nii.gz', '../database/retest_data/data/2SJU35U3.nii.gz', '../database/retest_data/data/3FEL3BY3.nii.gz', '../database/retest_data/data/4AIP225X.nii.gz', '../database/retest_data/data/0GDVZHBP.nii.gz', '../database/retest_data/data/0QYZKP52.

## 以上打印出来的就是BraTS19训练集中新增的那些病例，我们把它们作为测试集

In [52]:
import SimpleITK as sitk
import cv2

In [53]:
def normalize(slice, bottom=99, down=1):
    """
    normalize image with mean and std for regionnonzero,and clip the value into range
    :param slice:
    :param bottom:
    :param down:
    :return:
    """
    #有点像“去掉最低分去掉最高分”的意思,使得数据集更加“公平”
    b = np.percentile(slice, bottom)
    t = np.percentile(slice, down)
    slice = np.clip(slice, t, b)#限定范围numpy.clip(a, a_min, a_max, out=None)

    #除了黑色背景外的区域要进行标准化
    image_nonzero = slice[np.nonzero(slice)]
    if np.std(slice) == 0 or np.std(image_nonzero) == 0:
        return slice
    else:
        tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
        # since the range of intensities is between 0 and 5000 ,
        # the min in the normalized slice corresponds to 0 intensity in unnormalized slice
        # the min is replaced with -9 just to keep track of 0 intensities
        # so that we can discard those intensities afterwards when sampling random patches
        tmp[tmp == tmp.min()] = -9 #黑色背景区域
        return tmp

In [44]:
for subsetindex in range(len(pathmri_list)):
    print("原始标签",pathmri_list[subsetindex])
    print("新标签",subsetindex)
    data_image = pathmri_list[subsetindex]
    
    data_src = sitk.ReadImage(data_image, sitk.sitkInt16)
    #GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    data_array = sitk.GetArrayFromImage(data_src)
    
    data_array_nor = normalize(data_array)
    for index in range(0,90):
        index_1 = index - 1
        index_1_2 = index - 2
        index_2 = index + 1
        index_2_2 = index + 2
        if (index == 0):
            FiveImageArray = cv2.merge([data_array_nor[2],data_array_nor[1],data_array_nor[0],data_array_nor[1],data_array_nor[2]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,FiveImageArray)#(160,160,4) np.float dtype('float64')
        if (index == 1):
            FiveImageArray = cv2.merge([data_array_nor[1],data_array_nor[0],data_array_nor[1],data_array_nor[2],data_array_nor[3]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,FiveImageArray)#(160,160,4) np.float dtype('float64')
        if (index == 88):
            FiveImageArray = cv2.merge([data_array_nor[86],data_array_nor[87],data_array_nor[88],data_array_nor[89],data_array_nor[88]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,FiveImageArray)#(160,160,4) np.float dtype('float64')
        elif (index == 89):
            FiveImageArray = cv2.merge([data_array_nor[87],data_array_nor[88],data_array_nor[89],data_array_nor[88],data_array_nor[87]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,FiveImageArray)
        else:
            FiveImageArray = cv2.merge([data_array_nor[index_1_2],data_array_nor[index_1],data_array_nor[index],data_array_nor[index_2],data_array_nor[index_2_2]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,FiveImageArray)
        maskImg = mask_array[index,:,:]
        maskpath = outputMask_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(maskpath,maskImg)# 
        
print("Done！")

原始标签 ../database/retest3/00044986.nii.gz
新标签 0
原始标签 ../database/retest3/00024830.nii.gz
新标签 1
原始标签 ../database/retest3/00049658.nii.gz
新标签 2
原始标签 ../database/retest3/00049046.nii.gz
新标签 3
原始标签 ../database/retest3/00049799.nii.gz
新标签 4
原始标签 ../database/retest3/00046828.nii.gz
新标签 5
原始标签 ../database/retest3/00052242.nii.gz
新标签 6
原始标签 ../database/retest3/00049228.nii.gz
新标签 7
原始标签 ../database/retest3/28883.nii.gz
新标签 8
原始标签 ../database/retest3/00018374.nii.gz
新标签 9
原始标签 ../database/retest3/00047281.nii.gz
新标签 10
原始标签 ../database/retest3/00051842.nii.gz
新标签 11
原始标签 ../database/retest3/00049168.nii.gz
新标签 12
原始标签 ../database/retest3/00050448.nii.gz
新标签 13
原始标签 ../database/retest3/00048982.nii.gz
新标签 14
原始标签 ../database/retest3/00047161.nii.gz
新标签 15
原始标签 ../database/retest3/29259.nii.gz
新标签 16
原始标签 ../database/retest3/00041451.nii.gz
新标签 17
原始标签 ../database/retest3/48647.nii.gz
新标签 18
原始标签 ../database/retest3/00045872.nii.gz
新标签 19
原始标签 ../database/retest3/00050304.nii.gz
新标签 20
原始标签 ../dat

In [18]:
for subsetindex in range(len(pathmri_list)):
    print("原始标签",pathmri_list[subsetindex])
    print("新标签",subsetindex)
    flair_image = pathmri_list[subsetindex]
    mask_image = pathmask_list[subsetindex]
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    mask_array = sitk.GetArrayFromImage(mask)
    flair_array_nor = normalize(flair_array)
    for index in range(0,90):
        index_1 = index - 1
        index_1_2 = index - 2
        index_2 = index + 1
        index_2_2 = index + 2
        if (index == 0):
            ThreeImageArray = cv2.merge([flair_array_nor[1],flair_array_nor[0],flair_array_nor[1]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        elif (index == 89):
            ThreeImageArray = cv2.merge([flair_array_nor[88],flair_array_nor[89],flair_array_nor[88]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)
        else:
            ThreeImageArray = cv2.merge([flair_array_nor[index_1],flair_array_nor[index],flair_array_nor[index_2]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)
        imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(imagepath,ThreeImageArray)
        maskImg = mask_array[index,:,:]
        maskpath = outputMask_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(maskpath,maskImg)# (160, 160) dtype('uint8') 值为0 1 2 4
print("Done！")

原始标签 ../database/retest_data/data/3GP1RSTO.nii.gz
新标签 0
原始标签 ../database/retest_data/data/3CT52UTW.nii.gz
新标签 1
原始标签 ../database/retest_data/data/1RYHNWAY.nii.gz
新标签 2
原始标签 ../database/retest_data/data/2WUKD0FS.nii.gz
新标签 3
原始标签 ../database/retest_data/data/2KEL021V.nii.gz
新标签 4
原始标签 ../database/retest_data/data/0ESRE0DB.nii.gz
新标签 5
原始标签 ../database/retest_data/data/0ET3UZSR.nii.gz
新标签 6
原始标签 ../database/retest_data/data/2KWIPQOS.nii.gz
新标签 7
原始标签 ../database/retest_data/data/0QCYIPQQ.nii.gz
新标签 8
原始标签 ../database/retest_data/data/1KCPYDVM.nii.gz
新标签 9
原始标签 ../database/retest_data/data/4DEQNVOE.nii.gz
新标签 10
原始标签 ../database/retest_data/data/1TCAPQBI.nii.gz
新标签 11
原始标签 ../database/retest_data/data/0JNTNNBN.nii.gz
新标签 12
原始标签 ../database/retest_data/data/4GC2RPXG.nii.gz
新标签 13
原始标签 ../database/retest_data/data/1FRWVMWS.nii.gz
新标签 14
原始标签 ../database/retest_data/data/1R0TXANO.nii.gz
新标签 15
原始标签 ../database/retest_data/data/2SJU35U3.nii.gz
新标签 16
原始标签 ../database/retest_data/data/3FEL3BY

In [54]:
for subsetindex in range(len(pathmri_list)):
    print("原始标签",pathmri_list[subsetindex])
    print("新标签",subsetindex)
    flair_image = pathmri_list[subsetindex]
    mask_image = pathmask_list[subsetindex]
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    mask_array = sitk.GetArrayFromImage(mask)
    flair_array_nor = normalize(flair_array)
    for index in range(0,90):
        ThreeImageArray = cv2.merge([flair_array_nor[index],flair_array_nor[index]])
        imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(imagepath,ThreeImageArray)
        
        maskImg = mask_array[index,:,:]
        maskpath = outputMask_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(maskpath,maskImg)# (160, 160) dtype('uint8') 值为0 1 2 4
print("Done！")

原始标签 ../database/retest_data/data/3GP1RSTO.nii.gz
新标签 0
原始标签 ../database/retest_data/data/3CT52UTW.nii.gz
新标签 1
原始标签 ../database/retest_data/data/1RYHNWAY.nii.gz
新标签 2
原始标签 ../database/retest_data/data/2WUKD0FS.nii.gz
新标签 3
原始标签 ../database/retest_data/data/2KEL021V.nii.gz
新标签 4
原始标签 ../database/retest_data/data/0ESRE0DB.nii.gz
新标签 5
原始标签 ../database/retest_data/data/0ET3UZSR.nii.gz
新标签 6
原始标签 ../database/retest_data/data/2KWIPQOS.nii.gz
新标签 7
原始标签 ../database/retest_data/data/0QCYIPQQ.nii.gz
新标签 8
原始标签 ../database/retest_data/data/1KCPYDVM.nii.gz
新标签 9
原始标签 ../database/retest_data/data/4DEQNVOE.nii.gz
新标签 10
原始标签 ../database/retest_data/data/1TCAPQBI.nii.gz
新标签 11
原始标签 ../database/retest_data/data/0JNTNNBN.nii.gz
新标签 12
原始标签 ../database/retest_data/data/4GC2RPXG.nii.gz
新标签 13
原始标签 ../database/retest_data/data/1FRWVMWS.nii.gz
新标签 14
原始标签 ../database/retest_data/data/1R0TXANO.nii.gz
新标签 15
原始标签 ../database/retest_data/data/2SJU35U3.nii.gz
新标签 16
原始标签 ../database/retest_data/data/3FEL3BY

In [60]:
for subsetindex in range(len(pathmri_list)):
    print("原始标签",pathmri_list[subsetindex])
    print("新标签",subsetindex)
    flair_image = pathmri_list[subsetindex]
    mask_image = pathmask_list[subsetindex]
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    mask_array = sitk.GetArrayFromImage(mask)
    flair_array_nor = normalize(flair_array)
    for index in range(0,90):
        index_1 = index - 1
        index_1_2 = index - 2
        index_1_3 = index - 3
        index_2 = index + 1
        index_2_2 = index + 2
        index_2_3 = index + 3
        if (index == 0):
            ThreeImageArray = cv2.merge([flair_array_nor[3],flair_array_nor[2],flair_array_nor[1],flair_array_nor[0],flair_array_nor[1],flair_array_nor[2],flair_array_nor[3]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)
        if (index == 1):
            ThreeImageArray = cv2.merge([flair_array_nor[2],flair_array_nor[1],flair_array_nor[0],flair_array_nor[1],flair_array_nor[2],flair_array_nor[3],flair_array_nor[4]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)
        if (index == 2):
            ThreeImageArray = cv2.merge([flair_array_nor[1],flair_array_nor[0],flair_array_nor[1],flair_array_nor[2],flair_array_nor[3],flair_array_nor[4],flair_array_nor[5]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        if (index == 87):
            ThreeImageArray = cv2.merge([flair_array_nor[84],flair_array_nor[85],flair_array_nor[86],flair_array_nor[87],flair_array_nor[88],flair_array_nor[89],flair_array_nor[88]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        if (index == 88):
            ThreeImageArray = cv2.merge([flair_array_nor[85],flair_array_nor[86],flair_array_nor[87],flair_array_nor[88],flair_array_nor[89],flair_array_nor[88],flair_array_nor[87]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        if (index == 89):
            ThreeImageArray = cv2.merge([flair_array_nor[86],flair_array_nor[87],flair_array_nor[88],flair_array_nor[89],flair_array_nor[88],flair_array_nor[87],flair_array_nor[86]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        else:
            ThreeImageArray = cv2.merge([flair_array_nor[index_1_3],flair_array_nor[index_1_2],flair_array_nor[index_1],flair_array_nor[index],flair_array_nor[index_2],flair_array_nor[index_2_2],flair_array_nor[index_2_3]])
            imagepath = outputImg_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
            np.save(imagepath,ThreeImageArray)#(160,160,4) np.float dtype('float64')
        maskImg = mask_array[index,:,:]
        maskpath = outputMask_path + "//" + str(subsetindex) + "_" + str(index) + ".npy"
        np.save(maskpath,maskImg)# (160, 160) dtype('uint8') 值为0 1 2 4
print("Done！")

原始标签 ../database/retest_data/data/3GP1RSTO.nii.gz
新标签 0


IndexError: index 90 is out of bounds for axis 0 with size 90